>This code is responsible for constructing a dataframe for director scores based on revenue in movies directed and previous 2 years' director score

In [1]:
!pip install tmdbv3api

In [1]:
from tmdbv3api import TMDb
from tmdbv3api import Movie
from tmdbv3api.exceptions import TMDbException
import random
import pandas as pd
import csv
import numpy as np
from math import exp
import ast
tmdb=TMDb()
tmdb.api_key='9cf68f4c97c8f0cc6bb9646da389a808'

In [4]:
# Initialize csv file paths
csv_file_path1='merged_data_with_2010_data.csv'
# Create DataFrames from both CSV files
df1 = pd.read_csv(csv_file_path1)

>This next cell constructs the director dataframe of all unique director ids. For now, the star_score value is 0. 

In [6]:
import pandas as pd
import ast  # Import the ast module
import numpy as np

# Create a DataFrame with unique director IDs
director_ids = []

for index, row in df1.iterrows():
    director_ids_str = row['director_ids']
    
    # Check for NaN values and skip them
    if pd.isna(director_ids_str):
        continue

    # Safely evaluate the content of 'director_ids' if it's not NaN
    director_ids += ast.literal_eval(director_ids_str)

unique_director_ids = list(set(director_ids))

# Create a list of years from 2011 to 2023
years = [str(year) for year in range(2011, 2024)]

# Initialize the data with zeros
data = {f'star_{year}': [0] * len(unique_director_ids) for year in years}
data['ids'] = unique_director_ids

# Create the 'output_df' DataFrame
df2 = pd.DataFrame(data)

# Reorder columns with 'ids' as the first column
df2 = df2[['ids'] + [col for col in df2.columns if col != 'ids']]

# Print the first few rows of the 'output_df' DataFrame for debugging
print(df2.head())

       ids  star_2011  star_2012  star_2013  star_2014  star_2015  star_2016  \
0   229378          0          0          0          0          0          0   
1  1179651          0          0          0          0          0          0   
2   229380          0          0          0          0          0          0   
3        7          0          0          0          0          0          0   
4        8          0          0          0          0          0          0   

   star_2017  star_2018  star_2019  star_2020  star_2021  star_2022  star_2023  
0          0          0          0          0          0          0          0  
1          0          0          0          0          0          0          0  
2          0          0          0          0          0          0          0  
3          0          0          0          0          0          0          0  
4          0          0          0          0          0          0          0  


In [60]:
# Iterate through df1 to calculate director scores
for index, row in df1.iterrows():
    director_ids_str = row['director_ids']

    
    
    # Check for NaN values and skip them
    if pd.isna(director_ids_str):
        continue
    # Safely evaluate the content of 'director_ids' if it's not NaN
    director_ids = ast.literal_eval(director_ids_str)
    release_year = row['Release Year']
    merged_revenue = row['Merged Revenue']
    # Iterate through each director_id in the director_ids array
    for director_id in director_ids:
        # Find the corresponding row in df2
        df2_row = df2[df2['ids'] == director_id]
        if not df2_row.empty:
            director_score = df2_row[f'star_{release_year}'].values[0]
            # Calculate points for every million dollars in USD in directed movies
            if not pd.isna(merged_revenue):
                director_score += merged_revenue
            # Assign the updated director score to the corresponding 'star_yyyy' column
            df2.loc[df2['ids'] == director_id, f'star_{release_year}'] = director_score

for director_id in df2['ids']:
    for column in df2.columns:
        if column.startswith("star_"):
            release_year = int(column.split("_")[1])
            if release_year > 2011:
                previous_year = release_year - 1
                year_before_previous = release_year - 2
                df2_row = df2[df2['ids'] == director_id]
                if not df2_row.empty:
                    director_score = df2_row[column].values[0]
                    # Calculate score from the previous year
                    previous_year_score = df2_row[f'star_{previous_year}'].values[0]
                    director_score += previous_year_score
                    # Calculate score from the year before the previous year
                    if release_year > 2012:
                        year_before_previous_score = df2_row[f'star_{year_before_previous}'].values[0]
                        director_score += year_before_previous_score
                    # Assign the updated director score to the corresponding 'star_year' column
                    df2.loc[df2['ids'] == director_id, column] = director_score
# Create a dictionary to store rankings for each year
rankings = {}

# Calculate rankings for each year from 2011 to 2023
for year in range(2011, 2023):
    rank_column_name = f'star_{year}'
    # Get the top 100 revenues for the current year
    top_100 = df2[rank_column_name].nlargest(100).index
    # Set the ranking for the top 100 to range from 100 to 1
    rankings[year] = {index: 100 - i for i, index in enumerate(top_100)}
# Apply rankings to the 'star_yyyy' columns
for year, rank_map in rankings.items():
    rank_column_name = f'star_{year}'
    df2[rank_column_name] = df2.index.map(rank_map).fillna(0)
# Print the first few rows of the updated df2
print(df2.head())

       ids  star_2011  star_2012  star_2013  star_2014  star_2015  star_2016  \
0   229378      100.0      100.0        0.0        0.0        0.0        0.0   
1  1179651       99.0       99.0        0.0        0.0        0.0        0.0   
2   229380       98.0       98.0        0.0        0.0        0.0        0.0   
3        7       97.0       97.0        0.0        0.0        0.0       92.0   
4        8       96.0       96.0        0.0        0.0        0.0        0.0   

   star_2017  star_2018  star_2019  star_2020  star_2021  star_2022  star_2023  
0        0.0        0.0        0.0        0.0        0.0        0.0        0.0  
1        0.0        0.0        0.0        0.0        0.0        0.0        0.0  
2        0.0        0.0        0.0        0.0        0.0        0.0        0.0  
3       66.0       73.0       67.0       67.0       67.0       64.0       61.0  
4       26.0        0.0        0.0        0.0        0.0        0.0        0.0  


In [61]:
# Create an output dataframe
df_output = df2
# Save to csv
df_output.to_csv('director_df.csv', index=False)

In [63]:
star_scores = []  # Create a list to collect director scores
for index, row in df1.iterrows():
    row_dir_info = row['director_ids']  # Extract director_ids array
    if not pd.isna(row_dir_info):  # Check for NaN
        row_dir_info = ast.literal_eval(row_dir_info)
    else:
        row_dir_info = []  # Set to an empty list if NaN
    star_scores_row = []
    for director_id in row_dir_info:
        release_date = row['Release Year']
        star_column_name = f'star_{release_date}'  # Create the corresponding column name
        dir_info_row = df2[df2['ids'] == director_id]  # Get the row containing the queried director id
        while star_column_name not in dir_info_row and release_date > df1['Release Year'].min():
            release_date -= 1
            star_column_name = f'star_{release_date}'  # Try the previous year
        star_score = dir_info_row[star_column_name].values[0] if star_column_name in dir_info_row else None  # Use the current year if the previous isn't available
        star_scores_row.append(star_score)
    star_scores.append(star_scores_row)

# Create an output dataframe
df_output_1 = df1.copy()  # Copy the original DataFrame
df_output_1['director_scores'] = star_scores  # Add the 'star_scores' column

# Save to csv
print(df_output_1.head())
df_output_1.to_csv('merged_data_mod_star_score.csv', index=False)


   Unnamed: 0 Running Time                               Genres  \
0           0  1 hr 22 min                    Documentary Sport   
1           1        8 min                        History Short   
2           2  1 hr 49 min  Action Crime Drama Mystery Thriller   
3           3  1 hr 30 min                                Drama   
4           4  1 hr 46 min          Biography Drama History War   

                      IMDB Title   MPAA   Domestic Distributor  \
0           Into the Mind (2013)    NaN                    NaN   
1  In Search of Lost Time (2013)    NaN                    NaN   
2             Broken City (2013)      R  Twentieth Century Fox   
3         Redwood Highway (2013)  PG-13         Monterey Media   
4        Bless Me, Ultima (2012)  PG-13   Arenas Entertainment   

   Domestic Opening              Earliest Release Date  IMDB Budget  \
0               NaN           January 31, 2014 (Spain)            0   
1               NaN   September 30, 2007 (South Korea)    